In [1]:
from paik.solver import Solver, DEFAULT_SOLVER_PARAM_M7_NORM

solver = Solver(solver_param=DEFAULT_SOLVER_PARAM_M7_NORM)

WorldModel::LoadRobot: /home/luca/.cache/jrl/temp_urdfs/panda_arm_hand_formatted_link_filepaths_absolute.urdf
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link0.dae (59388 verts, 20478 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link1.dae (37309 verts, 12516 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link2.dae (37892 verts, 12716 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link3.dae (42512 verts, 14233 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link4.dae (43520 verts, 14620 tris)
LoadAssimp: Loaded model /ho

In [2]:
solver.shrink_ratio = 0.25
NUM_POSES = 10
NUM_SOLS = 1000
avg_l2, avg_ang, avg_inference_time, success_rate = solver.random_sample_solutions_with_evaluation_loop(NUM_POSES, NUM_SOLS)  # type: ignore
avg_l2, avg_ang, avg_inference_time, success_rate

100%|██████████| 10/10 [00:00<00:00, 10.57it/s]

                 l2           ang
count  10000.000000  10000.000000
mean       0.003076      1.097826
std        0.002318      2.467859
min        0.000068      0.051247
25%        0.001799      0.567999
50%        0.002729      0.881847
75%        0.003849      1.291042
max        0.061123    167.411756


(0.003076315721192414, 0.019160681447963786, 0.105, 0.0)

In [ ]:
import torch

path_solver = "/home/luca/pafik/weights/panda/1202-1325.pth"
state = torch.load(path_solver)
state["solver"]["base._0"]

In [ ]:
state = torch.load("solver.pth")
state["solver"]["base._0"]

In [ ]:
solver._solver.load_state_dict(state["solver"])

In [ ]:
from zuko.flows.spline import NSF
from paik.model import change_flow_base

flow = NSF(
    features=7,
    context=9,
    transforms=8,
    hidden_features=[1024] * 3,
    randperm=False,
    bins=10,
)

flow = change_flow_base(
    flow=flow,
    n=solver._n,
    shrink_ratio=0.25,
)
flow = flow.to("cuda")

In [ ]:
flow.load_state_dict(state["solver"])

In [ ]:
# try:
#     state = torch.load(path_solver)
#     # print(state["solver"])
#     flow.transforms.load_state_dict(state_dict=state["transforms"])
#     optimizer.load_state_dict(state_dict=state["opt"])
#     print(f"[SUCCESS] model load (transforms) from {path_solver}.")
# except Exception as e:
#     try:
#         # torch.save(
#         #     {
#         #         "transforms": flow.transforms.state_dict(),
#         #         "opt": optimizer.state_dict(),
#         #     },
#         #     path_solver,
#         # )
#         state = torch.load(path_solver)
#         flow.load_state_dict(state_dict=state["solver"])
#         optimizer.load_state_dict(state_dict=state["opt"])
#         print(f"[SUCCESS] model load (solver) from {path_solver}.")
#     except Exception as e1:
#         print(f"[WARNING]{e} and {e1}")

In [ ]:
from paik.file import remove_files_under_dir

remove_files_under_dir("./weights/panda/", except_files=["1202-1325"])

In [ ]:
C_denorm = solver.denorm_C(C_norm)
dfc = pd.DataFrame(C)
dfd = pd.DataFrame(C_denorm)
dfd.describe(), dfc.describe()

In [ ]:
mean_F = solver._F.mean(axis=0)
mean_P = solver._P_tr.mean(axis=0)
import numpy as np

C = np.column_stack([solver._P_tr, solver._F, np.zeros((solver._P_tr.shape[0], 1))])
mean_C = np.concatenate([mean_P, mean_F, np.zeros((1))])
mean_C, C.mean(axis=0)

In [ ]:
r_scale = 1.3
m = 7
r = 1
scale = np.ones_like(mean_C)
scale[m : m + r] *= r_scale
std_C = C.std(axis=0)
sstd = std_C / scale
print(C.shape, mean_C.shape, sstd.shape)
print(sstd)
C_norm_scale = (C - mean_C) / sstd
C_norm_scale.mean(axis=0), C_norm_scale.std(axis=0)

In [ ]:
import torch

# C_norm = (C - mean) / std
# C_norm_scale = C_norm * scale
# C_norm_scale = (C - mean) / std * scale
# sstd = std / scale
# C_norm_scale = (C - mean) / sstd
C_norm = solver.norm_C(C)
C_norm = C_norm * torch.tensor([1] * 7 + [1.3] * 1 + [1] * 1).to(C_norm.device)
C_norm.std(axis=0)

In [ ]:
import numpy as np
from dataclasses import dataclass


@dataclass
class DemoState:
    counter: int = 3
    target_pose: np.ndarray = np.array([0.5, 0.5, 0.5, 0, 0, 0])
    direction: bool = True


demo = DemoState()
demo

In [ ]:
import torch

In [ ]:
C = torch.ones((3, 2))
esp = 0.01
std_scale = 0.3
rand = torch.rand((C.shape[0], 1))

In [ ]:
s = esp * rand
CC = torch.column_stack((C, std_scale * s))
CC

In [ ]:
s = std_scale * esp * rand
CCc = torch.column_stack((C, s))
CCc

In [ ]:
import numpy as np

a = np.array([[3, 7, 18], [2, 4, 71]])

np.max(np.diff(a, axis=0))

In [ ]:
import torch

a = torch.tensor([[3, 7, 18], [2, 4, 71]])
aa = torch.repeat_interleave(a, 2, axis=0)
aaa = a.repeat(2, 1)
aaaa = a.view(1, 2, 3).tile(2, 1, 1)
aa, aaa, aaaa, *a.shape

In [ ]:
import itertools
import numpy as np
from pprint import pprint

n = 3
bounds = np.array([0.03, 0.03, 0.15])
base_position = bounds / 2 + np.array([-0.03, -0.15, 0.0])
get_bounds = lambda truth: [bounds[i] if truth[i] else 0 for i in range(n)]
table = list(itertools.product([False, True], repeat=n))
get_pos = lambda truth: tuple(base_position + get_bounds(truth))

for i in range(len(table)):
    for x in get_pos(table[i]):
        print(f"{x:.6f}", end=" ")
    print()

In [ ]:
batch_size = 3
batch = (torch.ones(batch_size, 7), torch.ones(batch_size, 7))
J, P = add_noise(batch, esp=0.1)
J.shape, P.shape

In [ ]:
s = 0.1 * torch.rand((J.shape[0], 1)).to(J.device)
noise = torch.normal(
    mean=torch.zeros_like(input=J),
    std=torch.repeat_interleave(input=s, repeats=J.shape[1], dim=1),
)

noise, torch.repeat_interleave(input=s, repeats=J.shape[1], dim=1)

In [ ]:
solver_param = DEFAULT_SOLVER_PARAM_M7
solver_param["model_architecture"] = "unaf"
solver_param

In [ ]:
solver = Solver(solver_param)

In [ ]:
solver._solver

In [ ]:
import torch
from zuko.flows import Flow, MaskedAutoregressiveTransform, Unconditional
from zuko.distributions import DiagNormal
from zuko.transforms import RotationTransform

ikflow_setttings = {
    "n_features": 7,
    "n_contexts": 7,
    "n_hidden_width": 1024,
    "n_hidden_depth": 3,
    "n_layers": 2,
    "shrink_ratio": 0.1,
}


def get_nf(n_features, n_contexts, n_hidden_width, n_hidden_depth, n_layers, **kwargs):
    return Flow(
        transforms=[
            MaskedAutoregressiveTransform(
                n_features,
                n_contexts,
                hidden_features=[n_hidden_width] * n_hidden_depth,
            )
            for _ in range(n_layers)
            # Unconditional(RotationTransform, torch.randn(n_features, n_features)), # type: ignore
        ],
        base=Unconditional(
            DiagNormal,
            torch.zeros(n_features),
            torch.ones(n_features),
            buffer=True,
        ),  # type: ignore
    )


flow = get_nf(**ikflow_setttings)
flow(torch.randn(1, 7)).sample()

In [ ]:
robot = Robot(verbose=False)
J_tr, P_tr, P_ts, F = load_all_data(robot, enable_normalize=False)
# knn = get_knn(P_tr=P_t

In [ ]:
config = {
    "subnet_width": 1400,
    "subnet_num_layers": 3,
    "num_transforms": 9,
    "lr": 2.1e-4,
    "lr_weight_decay": 2.7e-2,
    "decay_step_size": 4e4,
    "gamma": 5e-2,
    "batch_size": 128,
    "num_epochs": 10,
}

solver, optimizer, scheduler = get_flow_model(
    enable_load_model=cfg.use_pretrained,
    num_transforms=config["num_transforms"],
    subnet_width=config["subnet_width"],
    subnet_num_layers=config["subnet_num_layers"],
    lr=config["lr"],
    lr_weight_decay=config["lr_weight_decay"],
    decay_step_size=config["decay_step_size"],
    gamma=config["gamma"],
    device="cuda",
)

# flow = NSF(
#             features=cfg.n,
#             context=cfg.num_conditions,
#             transforms=config["num_transforms"],
#             randperm=True,
#             bins=10,
#             activation=cfg.activation,
#             hidden_features=[config["subnet_width"]] * config["subnet_num_layers"],
#         ).to('cuda')

# state = torch.load(cfg.path_solver)
# flow.load_state_dict(state_dict=state['solver'])

In [ ]:
def null_space_motion_single(robot, q):
    """
    return null space motion = (1-Jt@J)@Jm with normalize

    Parameters
    ----------
    robot : Robot
        abstract class of Robot
    q : np.ndarray
        joint configuration

    Returns
    -------
    np.array
        normalized null space motion that maximize manipulability
    """
    jacob = robot.robot.jacob0(q)
    null_projection = 1 - jacob.T @ jacob
    null_space_policy = robot.robot.jacobm(q)
    null_space_motion = null_projection @ null_space_policy
    norm_motion = null_space_motion / np.linalg.norm(null_space_motion)
    return norm_motion.reshape((cfg.n))

In [ ]:
def null_space_motion_array(robot, qs):
    null_motion_array = np.zeros_like(qs)
    i = 0
    for q in tqdm(qs):
        null_motion_array[i] = null_space_motion_single(robot, q)
        i += 1
    return null_motion_array

In [ ]:
q = J_tr[0]

In [ ]:
NS = null_space_motion_single(robot, q)
NS

In [ ]:
F[0]

In [ ]:
NM = null_space_motion_array(robot, J_tr[:1000])
NM.shape

In [ ]:
i, j = np.random.randint(low=0, high=len(J_tr), size=2)
p, q = J_tr[i], J_tr[j]
p_n = null_space_motion(robot, p)
q_n = null_space_motion(robot, q)
p_n.shape, p_n, np.linalg.norm(p_n)

In [ ]:
# NUM_STEPS = 250_0000 // 2
NUM_STEPS = 2000

position_distance = np.empty((NUM_STEPS))
null_distance = np.empty((NUM_STEPS))

from tqdm import trange

for step in trange(NUM_STEPS):
    i, j = np.random.randint(low=0, high=len(J_tr), size=2)
    p, q = J_tr[i], J_tr[j]
    p_n = null_space_motion(robot, p)
    q_n = null_space_motion(robot, q)

    position_distance[step] = np.linalg.norm(p - q)
    null_distance[step] = np.linalg.norm(p_n - q_n)

In [ ]:
df = pd.DataFrame(
    {"position_distance": position_distance, "null_distance": null_distance}
)
plot = df.plot.scatter(x="position_distance", y="null_distance")

In [ ]:
robot.robot.jacob0(q)

In [ ]:
robot.robot.jacob0(q).shape

In [ ]:
robot.robot.jacobm(q)

In [ ]:
mu = torch.zeros(size=(cfg.n,))
NUM_DATA = 100
NUM_SAMPLES = 1000
df = test(
    robot=robot,
    P_ts=P_ts[:NUM_DATA],
    F=F,
    solver=solver,
    knn=knn,
    K=NUM_SAMPLES,
    print_report=True,
)

In [ ]:
iflow = get_iflow_model(solver, init_sample=mu, shrink_ratio=0.11)
test(
    robot=robot,
    P_ts=P_ts[:NUM_DATA],
    F=F,
    solver=iflow,
    knn=knn,
    K=NUM_SAMPLES,
    print_report=True,
)

In [ ]:
# public final void set(Matrix4f m1) {
# 	w = Math.sqrt(1.0 + m1.m00 + m1.m11 + m1.m22) / 2.0;
# 	double w4 = (4.0 * w);
# 	x = (m1.m21 - m1.m12) / w4 ;
# 	y = (m1.m02 - m1.m20) / w4 ;
# 	z = (m1.m10 - m1.m01) / w4 ;
# }


def rot2qua(R):
    w = np.sqrt(1.0 + R[0, 0] + R[1, 1] + R[2, 2]) / 2.0
    w4 = 4.0 * w

    x = (R[2, 1] - R[1, 2]) / w4
    y = (R[0, 2] - R[2, 0]) / w4
    z = (R[1, 0] - R[0, 1]) / w4

    return (w, x, y, z)

In [ ]:
rot2qua(T.R)

In [ ]:
i = 6
neigh_idx = knn.kneighbors([P_ts_norm[i, :3]], n_neighbors=2, return_distance=False)
neigh_idx = neigh_idx.flatten()
neigh_idx, P_tr_norm[neigh_idx], P_ts_norm[i]

In [ ]:
i = 6
neigh_idx = knn.kneighbors([P_ts[i, :3]], n_neighbors=2, return_distance=False)
neigh_idx = neigh_idx.flatten()
neigh_idx, P_tr[neigh_idx], P_ts[i]

In [ ]:
neigh_idx = [2078302, 1221060]
neigh_idx, P_tr[neigh_idx], P_ts[i]

In [ ]:
i = 6
neigh_idx = knn.kneighbors([P_ts[i, :3]], n_neighbors=2, return_distance=False)
neigh_idx = neigh_idx.flatten()
neigh_idx, P_tr[neigh_idx], P_ts[i]

In [ ]:
i = 0
robot.position_orientation_errors_Arr_Inputs([J_tr_norm[i]], P_tr_norm[i])

In [ ]:
from pprint import pprint


def show_stat(arr):
    min = arr.min(axis=0)
    mean = arr.mean(axis=0)
    max = arr.max(axis=0)
    pprint((min, mean, max))

In [ ]:
show_stat(F)

In [ ]:
robot.joint_min, robot.joint_max

In [ ]:
J_norm = normalize(J_tr)
J_denorm = denormalize(J_norm, robot.joint_min, robot.joint_max)

In [ ]:
J_tr.min(axis=0), J_tr.max(axis=0), J_tr.mean(axis=0)

In [ ]:
J_denorm.min(axis=0), J_denorm.max(axis=0), J_denorm.mean(axis=0)

In [ ]:
J_norm.max(axis=0)

In [ ]:
df = pd.DataFrame(P_tr)
df.min().to_numpy(), df.max().to_numpy()

In [ ]:
df = pd.DataFrame(J_tr)
df.min(), df.max()

In [ ]:
df.min().to_numpy(), df.max().to_numpy()

In [ ]:
train_loader = get_train_loader(J=J_tr, P=P_tr, F=F, device="cuda", batch_size=128)

# Build Generative model, NSF
# Neural spline flow (NSF) with 3 sample features and 5 context features
solver, optimizer, scheduler = get_flow_model(
    load_model=cfg.use_pretrained,
    num_transforms=7,
    subnet_width=1024,
    subnet_num_layers=3,
    lr=4.8e-4,
    lr_weight_decay=5e-3,
    decay_step_size=4e4,
    gamma=0.5,
    device="cuda",
)

In [ ]:
rand = np.random.randint(low=0, high=len(P_ts), size=cfg.num_eval_size)
_, position_errors, _, _ = test(
    robot=robot,
    P_ts=P_ts[rand],
    F=F,
    solver=solver,
    knn=knn,
    K=cfg.K,
    print_report=False,
)

In [ ]:
com_pos = np.zeros((100, 7))

In [ ]:
com_pos[:, :3]

In [ ]:
position_errors

In [ ]:
class Net(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
def train_cifar(config):
    net = Net(config["l1"], config["l2"])

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)

    # To restore a checkpoint, use `session.get_checkpoint()`.
    loaded_checkpoint = session.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
            model_state, optimizer_state = torch.load(
                os.path.join(loaded_checkpoint_dir, "checkpoint.pt")
            )
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    data_dir = os.path.abspath("./data")
    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs]
    )

    trainloader = torch.utils.data.DataLoader(
        train_subset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=8
    )
    valloader = torch.utils.data.DataLoader(
        val_subset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=8
    )

    for epoch in range(10):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print(
                    "[%d, %5d] loss: %.3f"
                    % (epoch + 1, i + 1, running_loss / epoch_steps)
                )
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        # Here we save a checkpoint. It is automatically registered with
        # Ray Tune and can be accessed through `session.get_checkpoint()`
        # API in future iterations.
        os.makedirs("my_model", exist_ok=True)
        torch.save((net.state_dict(), optimizer.state_dict()), "my_model/checkpoint.pt")
        checkpoint = Checkpoint.from_directory("my_model")
        session.report(
            {"loss": (val_loss / val_steps), "accuracy": correct / total},
            checkpoint=checkpoint,
        )
    print("Finished Training")

In [ ]:
def _test_accuracy(net, device="cpu"):
    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [ ]:
def test_best_model(best_result):
    best_trained_model = Net(best_result.config["l1"], best_result.config["l2"])
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    best_trained_model.to(device)

    checkpoint_path = os.path.join(
        best_result.checkpoint.to_directory(), "checkpoint.pt"
    )

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = best_trained_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print("Best trial test set accuracy: {}".format(correct / total))

In [ ]:
config = {
    "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16]),
}

In [ ]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):
    config = {
        "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16]),
    }
    scheduler = ASHAScheduler(max_t=max_num_epochs, grace_period=1, reduction_factor=2)

    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train_cifar),
            resources={"cpu": 2, "gpu": gpus_per_trial},
        ),
        tune_config=tune.TuneConfig(
            metric="loss",
            mode="min",
            scheduler=scheduler,
            num_samples=num_samples,
        ),
        param_space=config,
    )
    results = tuner.fit()

    best_result = results.get_best_result("loss", "min")

    print("Best trial config: {}".format(best_result.config))
    print("Best trial final validation loss: {}".format(best_result.metrics["loss"]))
    print(
        "Best trial final validation accuracy: {}".format(
            best_result.metrics["accuracy"]
        )
    )

    test_best_model(best_result)


main(num_samples=2, max_num_epochs=2, gpus_per_trial=0)

In [ ]:
print(f"#trials={len(result.trials)}")
print(f"time={time.time()-start_time}")
best_trial = result.get_best_trial("loss", "min", "all")
print("Best trial config: {}".format(best_trial.config))
print(
    "Best trial final validation loss: {}".format(
        best_trial.metric_analysis["loss"]["min"]
    )
)
print(
    "Best trial final validation accuracy: {}".format(
        best_trial.metric_analysis["accuracy"]["max"]
    )
)

best_trained_model = Net(2 ** best_trial.config["l1"], 2 ** best_trial.config["l2"])
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"
    if gpus_per_trial > 1:
        best_trained_model = nn.DataParallel(best_trained_model)
best_trained_model.to(device)

checkpoint_value = (
    getattr(best_trial.checkpoint, "dir_or_data", None) or best_trial.checkpoint.value
)
checkpoint_path = os.path.join(checkpoint_value, "checkpoint")

model_state, optimizer_state = torch.load(checkpoint_path)
best_trained_model.load_state_dict(model_state)

test_acc = _test_accuracy(best_trained_model, device)
print("Best trial test set accuracy: {}".format(test_acc))